In [1]:
import pandas as pd

In [2]:
dispo = pd.read_csv('U://DATA/OpenClassrooms/P9/DAN-P9-data/DisponibiliteAlimentaire_2017.csv')
pop = pd.read_csv('U://DATA/OpenClassrooms/P9/DAN-P9-data/Population_2000_2018.csv')
price = pd.read_csv('U://DATA/OpenClassrooms/P9/FAOSTAT_data_prix_tonne.csv')
stability = pd.read_csv('U://DATA/OpenClassrooms/P9/FAOSTAT_data_stabilité_politique.csv')


In [3]:
def CheckTable(df):
    print(df.isnull().sum())
    print(df.duplicated().sum())
    print(df.describe())
    print(df.dtypes)
    print(df.shape)

In [4]:
#CheckTable(dispo)
dispo.head(10)

,Code Domaine,Domaine,Code zone,Zone,Code Élément,Élément,Code Produit,Produit,Code année,Année,Unité,Valeur,Symbole,Description du Symbole
0,FBS,Nouveaux Bilans Alimentaire,2,Afghanistan,5511,Production,2511,Blé et produits,2017,2017,Milliers de tonnes,4281.0,S,Données standardisées
1,FBS,Nouveaux Bilans Alimentaire,2,Afghanistan,5611,Importations - Quantité,2511,Blé et produits,2017,2017,Milliers de tonnes,2302.0,S,Données standardisées
2,FBS,Nouveaux Bilans Alimentaire,2,Afghanistan,5072,Variation de stock,2511,Blé et produits,2017,2017,Milliers de tonnes,-119.0,S,Données standardisées
3,FBS,Nouveaux Bilans Alimentaire,2,Afghanistan,5911,Exportations - Quantité,2511,Blé et produits,2017,2017,Milliers de tonnes,0.0,S,Données standardisées
4,FBS,Nouveaux Bilans Alimentaire,2,Afghanistan,5301,Disponibilité intérieure,2511,Blé et produits,2017,2017,Milliers de tonnes,6701.0,S,Données standardisées
5,FBS,Nouveaux Bilans Alimentaire,2,Afghanistan,5521,Aliments pour animaux,2511,Blé et produits,2017,2017,Milliers de tonnes,76.0,S,Données standardisées
6,FBS,Nouveaux Bilans Alimentaire,2,Afghanistan,5527,Semences,2511,Blé et produits,2017,2017,Milliers de tonnes,344.0,S,Données standardisées
7,FBS,Nouveaux Bilans Alimentaire,2,Afghanistan,5123,Pertes,2511,Blé et produits,2017,2017,Milliers de tonnes,642.0,S,Données standardisées
8,FBS,Nouveaux Bilans Alimentaire,2,Afghanistan,5170,Résidus,2511,Blé et produits,2017,2017,Milliers de tonnes,0.0,S,Données standardisées
9,FBS,Nouveaux Bilans Alimentaire,2,Afghanistan,5142,Nourriture,2511,Blé et produits,2017,2017,Milliers de tonnes,5640.0,S,Données standardisées


In [5]:
dispo['Unité'].unique()

array(['Milliers de tonnes', 'kg', 'Kcal/personne/jour',
       'g/personne/jour'], dtype=object)

**Cleaning the data**

On enlève les colones Code qui sont redondante et on ne garde que les produits qui nous interesse pour notre export de poulet : Viande de Volailles 

In [6]:
dispo_clean = dispo
dispo_clean.drop(['Code Domaine', 'Code zone', 'Domaine','Code Élément', 'Code Produit', 'Code année','Année','Symbole','Description du Symbole'], axis=1, inplace=True)
dispo_clean = dispo_clean[dispo_clean['Produit'].isin(['Viande de Volailles'])]
dispo_clean = dispo_clean[dispo_clean['Élément'].isin(['Importations - Quantité'])]

In [7]:
dispo_clean[dispo_clean['Zone'].isin(['Inde'])]

,Zone,Élément,Produit,Unité,Valeur
76626,Inde,Importations - Quantité,Viande de Volailles,Milliers de tonnes,0.0


In [8]:
dispo_clean = dispo_clean.pivot_table(index='Zone', columns='Produit', values='Valeur')
dispo_clean.reset_index(inplace=True)
dispo_clean

Produit,Zone,Viande de Volailles
0,Afghanistan,29.0
1,Afrique du Sud,514.0
2,Albanie,38.0
3,Algérie,2.0
4,Allemagne,842.0
...,...,...
165,Émirats arabes unis,433.0
166,Équateur,0.0
167,États-Unis d'Amérique,123.0
168,Éthiopie,1.0


In [9]:
dispo_clean = dispo_clean.rename(columns={'Viande de Volailles':'Import Viande de Volailles (Mt)'})

**Join pop table, pour l'année 2017**

In [10]:
pop = pop[pop['Année'].isin([2017])]
pop = pop[['Zone', 'Année','Valeur']]
pop['Valeur'] = round(pop['Valeur']*1000,0)
pop['Valeur'] = pop['Valeur'].astype(int)
pop = pop.rename(columns={'Valeur':'Population'})
pop.drop(['Année'], axis=1, inplace=True)
pop

,Zone,Population
17,Afghanistan,36296113
36,Afrique du Sud,57009756
55,Albanie,2884169
74,Algérie,41389189
93,Allemagne,82658409
...,...,...
4333,Venezuela (République bolivarienne du),29402484
4352,Viet Nam,94600648
4371,Yémen,27834819
4390,Zambie,16853599


In [11]:
data_clean = pd.merge(dispo_clean, pop, on='Zone', how='left')
data_clean.set_index('Zone', inplace=True)


-------------------------------------------------------
Préparation des autres data

In [12]:
price.drop(['Domaine','Année','Unité'], axis=1, inplace= True)

In [13]:
price_poulet = price[price['Produit'].isin(['Poulets'])]
price_poulet = price_poulet.pivot_table(index='Zone', columns='Élément', values='Valeur')
price_poulet = price_poulet.rename(columns={'Importations - Quantité':'Quantité Importé Poulets', 'Importations - Valeur':'Importations Poulets Valeur'})
price_poulet['Prix/poulets'] = price_poulet['Quantité Importé Poulets'] / price_poulet['Importations Poulets Valeur']
price_poulet

Élément,Quantité Importé Poulets,Importations Poulets Valeur,Prix/poulets
Zone,,,
Afghanistan,18970.0,26227.0,0.723300
Afrique du Sud,397.0,7394.0,0.053692
Albanie,4572.0,3266.0,1.399878
Algérie,195.0,19051.0,0.010236
Allemagne,185661.0,193759.0,0.958206
...,...,...,...
Érythrée,141.0,287.0,0.491289
États-Unis d'Amérique,4872.0,8580.0,0.567832
Éthiopie,2.0,91.0,0.021978


In [14]:
price_poulet.drop(['Quantité Importé Poulets','Importations Poulets Valeur'], axis=1, inplace=True)

In [15]:
stability.drop(['Domaine','Élément','Produit','Année','Unité'], axis=1, inplace=True)
stability

,Zone,Valeur
0,Afghanistan,-2.80
1,Afrique du Sud,-0.28
2,Albanie,0.38
3,Algérie,-0.92
4,Allemagne,0.59
...,...,...
192,Venezuela (République bolivarienne du),-1.27
193,Viet Nam,0.23
194,Yémen,-2.94
195,Zambie,0.15


In [16]:
stability = stability.rename(columns={'Valeur':'Political Stability'})

In [17]:
data_all = pd.merge(price_poulet,data_clean, on='Zone')
data_all = pd.merge(stability,data_all, on='Zone')
data_all.set_index('Zone', inplace=True)
data_all['Ratio viande Importer / Population'] = data_all['Import Viande de Volailles (Mt)'] / data_all['Population']
data_all = data_all[data_all['Import Viande de Volailles (Mt)'] != 0]
data_all = data_all[data_all.index != 'Bosnie-Herzégovine']
data_all.head(10)

,Political Stability,Prix/poulets,Import Viande de Volailles (Mt),Population,Ratio viande Importer / Population
Zone,,,,,
Afghanistan,-2.80,0.723300,29.0,36296113,7.989836e-07
Afrique du Sud,-0.28,0.053692,514.0,57009756,9.016001e-06
Albanie,0.38,1.399878,38.0,2884169,1.317537e-05
Algérie,-0.92,0.010236,2.0,41389189,4.832180e-08
Allemagne,0.59,0.958206,842.0,82658409,1.018650e-05
Angola,-0.38,0.909041,277.0,29816766,9.290075e-06
Antigua-et-Barbuda,0.75,0.285714,7.0,95426,7.335527e-05
Arabie saoudite,-0.64,0.017491,722.0,33101179,2.181191e-05
Argentine,0.17,0.044913,8.0,43937140,1.820783e-07


In [18]:
data_all.sort_values('Import Viande de Volailles (Mt)').head()

,Political Stability,Prix/poulets,Import Viande de Volailles (Mt),Population,Ratio viande Importer / Population
Zone,,,,,
Bolivie (État plurinational de),-0.28,0.198011,1.0,11192855,8.934271e-08
Mali,-1.91,0.571159,1.0,18512430,5.401776e-08
Paraguay,0.00,1.274484,1.0,6867061,1.456227e-07
Éthiopie,-1.68,0.021978,1.0,106399924,9.398503e-09
Indonésie,-0.50,2.490688,1.0,264650963,3.778562e-09


In [19]:
no_pop = data_all.copy().drop(['Population'], axis=1)

In [20]:
no_stability = data_all.copy().drop(['Political Stability'], axis=1)

In [21]:
clear_data = data_all.copy().drop(['Population',], axis=1)
clear_data.drop(['Political Stability'], axis=1, inplace=True)
clear_data.drop(['Prix/poulets'], axis=1, inplace=True)
clear_data

,Import Viande de Volailles (Mt),Ratio viande Importer / Population
Zone,,
Afghanistan,29.0,7.989836e-07
Afrique du Sud,514.0,9.016001e-06
Albanie,38.0,1.317537e-05
Algérie,2.0,4.832180e-08
Allemagne,842.0,1.018650e-05
...,...,...
Venezuela (République bolivarienne du),25.0,8.502683e-07
Viet Nam,291.0,3.076089e-06
Yémen,78.0,2.802246e-06


In [22]:
%store data_all
%store no_pop
%store no_stability
%store data_clean
%store clear_data

Stored 'data_all' (DataFrame)
Stored 'no_pop' (DataFrame)
Stored 'no_stability' (DataFrame)
Stored 'data_clean' (DataFrame)
Stored 'clear_data' (DataFrame)


-----------------------------------------------------------------------
Next notebook for clustering and exploration
------------------------------------------------------------------------
----------------------------------